In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import mysql.connector
from sqlalchemy import create_engine
import pymysql
from datetime import datetime

In [2]:
url = "https://www.numbeo.com/cost-of-living/rankings_current.jsp"

In [3]:
response = requests.get(url).text

In [4]:
soup = BeautifulSoup(response,"html.parser")

In [5]:
tables = soup.find_all('table')
# tables

In [6]:
table = soup.find('table',class_ = "stripe row-border order-column compact")
# table

In [7]:
title = table.find_all('th')
title

[<th><div style="font-size: 80%; vertical-align: middle;">Rank</div></th>,
 <th><div style="font-size: 95%;">City</div></th>,
 <th><div style="font-size: 90%;">Cost of Living Index</div></th>,
 <th><div style="font-size: 90%;">Rent Index</div></th>,
 <th><div style="font-size: 90%;">Cost of Living Plus Rent Index</div></th>,
 <th><div style="font-size: 90%;">Groceries Index</div></th>,
 <th><div style="font-size: 90%;">Restaurant Price Index</div></th>,
 <th><div style="font-size: 90%;">Local Purchasing Power Index</div></th>]

In [8]:
header = []

for i in title:
    name = i.text
    header.append(name)
print(header)

['Rank', 'City', 'Cost of Living Index', 'Rent Index', 'Cost of Living Plus Rent Index', 'Groceries Index', 'Restaurant Price Index', 'Local Purchasing Power Index']


In [9]:
rows = table.tbody.select('tr')


In [10]:
data = [dict(zip(header,[cell.get_text(strip=True) for cell in row.find_all('td')])) for row in rows[0:]]

In [11]:
df = pd.DataFrame(data)


In [12]:
df[['Cost of Living Index','Rent Index','Cost of Living Plus Rent Index','Groceries Index','Restaurant Price Index','Local Purchasing Power Index']] = df[['Cost of Living Index','Rent Index','Cost of Living Plus Rent Index','Groceries Index','Restaurant Price Index','Local Purchasing Power Index']].astype('float')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Rank                            569 non-null    object 
 1   City                            569 non-null    object 
 2   Cost of Living Index            569 non-null    float64
 3   Rent Index                      569 non-null    float64
 4   Cost of Living Plus Rent Index  569 non-null    float64
 5   Groceries Index                 569 non-null    float64
 6   Restaurant Price Index          569 non-null    float64
 7   Local Purchasing Power Index    569 non-null    float64
dtypes: float64(6), object(2)
memory usage: 35.7+ KB


In [14]:
df['Rank'] = df['Cost of Living Index'].rank(ascending=False).astype('int64')
df

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,"Hamilton, Bermuda",142.0,92.2,117.9,147.0,139.4,78.0
1,2,"Zurich, Switzerland",128.9,67.5,99.2,122.3,120.7,111.0
2,3,"Basel, Switzerland",126.3,44.8,86.9,135.1,119.0,118.4
3,4,"Lausanne, Switzerland",121.7,52.8,88.3,118.2,112.1,103.4
4,5,"Zug, Switzerland",118.7,65.0,92.7,113.4,122.2,127.0
...,...,...,...,...,...,...,...,...
564,565,"Islamabad, Pakistan",16.8,4.4,10.8,14.3,12.2,26.1
565,566,"Multan, Pakistan",13.7,2.8,8.4,12.5,10.1,22.8
566,567,"Rawalpindi, Pakistan",13.5,2.3,8.1,12.1,10.4,27.2
567,568,"Faisalabad, Pakistan",12.8,2.2,7.6,10.0,8.0,30.9


In [15]:
links = []

url = "https://www.numbeo.com/cost-of-living/rankings_current.jsp"

r = requests.get(url).text

s = BeautifulSoup(r,'html.parser')


for i in s.find_all('a',class_="discreet_link"):
    a = i.get('href')
    links.append(a)



In [16]:
citieS = []

for City in df['City']:
    a = citieS.append(City)

In [17]:
my_dict = dict(zip(citieS,links[1:]))

In [18]:
def city_name(Url) :
    my_dict = dict(zip(citieS,links[1:]))
    
    # for url in links:
    for city,city_url in my_dict.items():
            # if city == "Hamilton, Bermuda":
            if Url == city_url:
                return city

In [19]:
def clean_string(original_string):
    
     # Initialize a counter for the number of decimal points
    decimal_count = 0
    
    # Keep only the numeric characters and the first decimal point
    clean_string = ''
    for c in original_string:
        
        if c.isnumeric():
            clean_string += c
            
        elif c == '.':
            decimal_count += 1
            
            if decimal_count == 1:
                clean_string += c
                
    # Check if the cleaned string is empty
    if not clean_string:
        # Return a default value
        return None
    
    else:
        # Convert the cleaned string to a float
        clean_float = float(clean_string)
        return clean_float

In [20]:
def move_column(df, column_name, position):
    col = df.pop(column_name) # remove column and store it in col
    df.insert(position, column_name, col) # insert column at specified position

In [21]:
final_restaurant_cost = []
final_market_cost = []
final_transportation_cost = [] 
final_utilities_monthly_cost = []
final_sports_and_leisure_cost = []
final_childcare_cost = []
final_clothing_and_shoes_cost = []
final_rent_per_month_cost = []
final_buy_apartment_price_cost = []
final_salaries_and_financing_cost = []

In [22]:

for url in links[1:]:
    
    response = requests.get(url).text
    soup = BeautifulSoup(response,"html.parser")
    new_table = soup.find('table',class_= "data_wide_table new_bar_table")
    ro = new_table.find_all('tr')
    header1 = ['Restaurants','Amount','Markets','Amount','Transportation','Amount','Utilities (Monthly)','Amount','Sports And Leisure','Amount','Childcare','Amount','Clothing And Shoes','Amount','Rent Per Month','Amount','Buy Apartment Price','Amount','Salaries And Financing','Amount']
    city_name1 = city_name(url)
    
    restaurant_cost = pd.DataFrame([dict(zip(header1[0:2],[cell.text for cell in row.find_all('td')])) for row in ro[1:9]])
    restaurant_cost['Amount']=restaurant_cost['Amount'].apply(clean_string)
    restaurant_cost["City"] = city_name1
    move_column(restaurant_cost,'City',0)
    final_restaurant_cost.append(restaurant_cost)

    market_cost = pd.DataFrame([dict(zip(header1[2:4],[cell.text for cell in row.find_all('td')])) for row in ro[10:29]])
    market_cost['Amount'] =  market_cost['Amount'].apply(clean_string)
    market_cost["City"] = city_name(url)
    move_column(market_cost,'City',0)
    final_market_cost.append(market_cost)
    
    transportation_cost = pd.DataFrame([dict(zip(header1[4:6],[cell.text for cell in row.find_all('td')])) for row in ro[30:38]])
    transportation_cost['Amount'] =  transportation_cost['Amount'].apply(clean_string)
    transportation_cost["City"] = city_name(url)
    move_column(transportation_cost,'City',0)
    final_transportation_cost.append(transportation_cost)
    
    utilities_monthly_cost = pd.DataFrame([dict(zip(header1[6:8],[cell.text for cell in row.find_all('td')])) for row in ro[39:42]])
    utilities_monthly_cost['Amount'] =  utilities_monthly_cost['Amount'].apply(clean_string)
    utilities_monthly_cost["City"] = city_name(url)
    move_column(utilities_monthly_cost,'City',0)
    final_utilities_monthly_cost.append(utilities_monthly_cost)
    
    sports_and_leisure_cost =  pd.DataFrame([dict(zip(header1[8:10],[cell.text for cell in row.find_all('td')])) for row in ro[43:46]])
    sports_and_leisure_cost['Amount'] =  sports_and_leisure_cost['Amount'].apply(clean_string)
    sports_and_leisure_cost["City"] = city_name(url)
    move_column(sports_and_leisure_cost,'City',0)
    final_sports_and_leisure_cost.append(sports_and_leisure_cost)
    
    childcare_cost = pd.DataFrame([dict(zip(header1[10:12],[cell.text for cell in row.find_all('td')])) for row in ro[47:49]])
    childcare_cost['Amount'] =  childcare_cost['Amount'].apply(clean_string)
    childcare_cost["City"] = city_name(url)
    move_column(childcare_cost,'City',0)
    final_childcare_cost.append(childcare_cost)
    
    clothing_and_shoes_cost = pd.DataFrame([dict(zip(header1[12:14],[cell.text for cell in row.find_all('td')])) for row in ro[50:54]])
    clothing_and_shoes_cost['Amount'] =  clothing_and_shoes_cost['Amount'].apply(clean_string)
    clothing_and_shoes_cost["City"] = city_name(url)
    move_column(clothing_and_shoes_cost,'City',0)
    final_clothing_and_shoes_cost.append(clothing_and_shoes_cost)
    
    rent_per_month_cost = pd.DataFrame([dict(zip(header1[14:16],[cell.text for cell in row.find_all('td')])) for row in ro[55:59]])
    rent_per_month_cost['Amount'] = rent_per_month_cost['Amount'].apply(clean_string)
    rent_per_month_cost['City'] = city_name(url)
    move_column(rent_per_month_cost,'City',0)
    final_rent_per_month_cost.append(rent_per_month_cost)

    buy_apartment_price_cost = pd.DataFrame([dict(zip(header1[16:18],[cell.text for cell in row.find_all('td')])) for row in ro[60:62]])
    buy_apartment_price_cost['Amount'] = buy_apartment_price_cost['Amount'].apply(clean_string)
    buy_apartment_price_cost["City"] = city_name(url)
    move_column(buy_apartment_price_cost,'City',0)
    final_buy_apartment_price_cost.append(buy_apartment_price_cost)
    
    salaries_and_financing_cost = pd.DataFrame([dict(zip(header1[18:20],[cell.text for cell in row.find_all('td')])) for row in ro[63:65]])
    salaries_and_financing_cost['Amount'] = salaries_and_financing_cost['Amount'].apply(clean_string)
    salaries_and_financing_cost["City"] = city_name(url)
    move_column(salaries_and_financing_cost,'City',0)
    final_salaries_and_financing_cost.append(salaries_and_financing_cost)


In [23]:
restaurant_expenses = pd.concat(final_restaurant_cost)
market_expenses = pd.concat(final_market_cost)
tran_expenses = pd.concat(final_transportation_cost)
utilities_expenses = pd.concat(final_utilities_monthly_cost)
sport_and_leisure_expenses = pd.concat(final_sports_and_leisure_cost)
childcare_expenses = pd.concat(final_childcare_cost)
clothing_expenses = pd.concat(final_clothing_and_shoes_cost)
rent_per_month = pd.concat(final_rent_per_month_cost)
buy_apartment = pd.concat(final_buy_apartment_price_cost)
salaries_expenses = pd.concat(final_salaries_and_financing_cost)

In [24]:
name = 'root'
host = 'localhost'
password = 'wahid786'
port = 3306
dbname = 'WEBSCRAPPING'

engine = create_engine(f'mysql+pymysql://{name}:{password}@{host}:{port}/{dbname}')

In [25]:
df.to_sql('cities',engine,if_exists = 'replace',index=False)

569

In [26]:
restaurant_expenses.to_sql('restaurant_cost_test',engine,if_exists = 'replace',index=False)

4552

In [27]:
market_expenses.to_sql('market_cost_set',engine,if_exists = 'replace',index=False)

10811

In [28]:
tran_expenses.to_sql('transportation_cost_status',engine,if_exists = 'replace',index=False)

4552

In [29]:
utilities_expenses.to_sql('utilies_cost_set',engine,if_exists = 'replace',index=False)

1707

In [30]:
sport_and_leisure_expenses.to_sql('spports_leisure_cost_set',engine,if_exists = 'replace',index=False)

1707

In [31]:
childcare_expenses.to_sql('childcare',engine,if_exists = 'replace',index=False)

1138

In [32]:
clothing_expenses.to_sql('clothing_shoes_cost_set',engine,if_exists = 'replace',index=False)

2276

In [33]:
rent_per_month.to_sql('rent_cost_set',engine,if_exists = 'replace',index=False)

2276

In [34]:
buy_apartment.to_sql('buy_apt_cost_set',engine,if_exists = 'replace',index=False)

1138

In [35]:
salaries_expenses.to_sql('salary_and_financing_cost',engine,if_exists = 'replace',index=False)

1138

In [8]:
file = open(r"C:\\Users\\hp\\OneDrive\\Desktop\\Python\\Projects\\webscrapping.txt",'a')
file.write(f'{datetime.datetime.now} - The Script ran\n')

76